In [1]:
import sys
import os
import pandas as pd

sys.path.append(os.path.abspath("../../ml/"))

from dev_wtp_io_utils import *

/home/bsmith16/.conda/envs/neuralsignature/lib/python3.8/site-packages/nilearn/datasets/__init__.py:87: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  warn("Fetchers from the nilearn.datasets module will be "


In [2]:
nonbids_data_path = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/"
ml_data_folderpath = "/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/ml"

In [3]:
test_train_df_raw = pd.read_csv(nonbids_data_path + "fMRI/ml/train_test_markers_20210601T183243.csv")


In [4]:
include_exclude_list = pd.read_csv("../nsc_subject_exclusions.csv")

In [5]:
test_train_df_raw = test_train_df_raw.merge(include_exclude_list[include_exclude_list.Task=='SST'],left_on='sub_label',right_on='SubjectId',how='left')
test_train_df_raw.loc[test_train_df_raw.Include.isna(),'Include'] = True
test_train_df = test_train_df_raw[test_train_df_raw.Include==True]

In [6]:
exclude_subjects = ['DEV061','DEV185','DEV187','DEV189','DEV190','DEV192','DEV198','DEV203','DEV220','DEV221']


In [7]:
train_subjs = test_train_df.loc[test_train_df.SplitGroup=='Train','sub_label'].tolist()#only get the train subjects; ignore those previously marked hold-out

In [8]:
test_train_df

,sub_label,SplitGroup,SubjectId,Task,Include
1,DEV004,Holdout,NaN,NaN,True
2,DEV005,Train,NaN,NaN,True
3,DEV006,Train,NaN,NaN,True
4,DEV008,Holdout,NaN,NaN,True
5,DEV009,Train,NaN,NaN,True
...,...,...,...,...,...
70,DEV085,Train,NaN,NaN,True
71,DEV086,Train,NaN,NaN,True
72,DEV087,Train,NaN,NaN,True
73,DEV089,Train,NaN,NaN,True


In [9]:
def get_Brain_Data_for_sub(sub_label,behavdesign,betaseries_path='/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/'):
    return(dev_wtp_io_utils.get_Brain_Data_betas_for_sub(sub_label,behavdesign,betaseries_path))

In [10]:
from dev_wtp_io_utils import import_sst_betaseries_w1_subjs_to_pkl

In [11]:
all_sst_events= pd.read_csv(ml_data_folderpath +"/SST/" + "all_sst_events.csv")
#cut out other waves
sst_1_behavdesign_1 = all_sst_events[(all_sst_events.wave=='wave1')].reset_index(drop=True)
#label the ITI as such
sst_1_behavdesign_1.loc[(all_sst_events.trial_type.isna()),'trial_type']='ITI'
#ensure it is correctly ordered
sst_1_behavdesign_clean = sst_1_behavdesign_1.sort_values(['subject','onset']).reset_index(drop=True)
#now tell it the name of the corresponding beta. this is easy; they are all consecutive
sst_1_behavdesign_clean['beta'] = 'beta_' + (sst_1_behavdesign_clean.groupby(['subject','wave']).cumcount()+1).astype(str).apply(lambda x: x.zfill(4)) + '.nii'

In [12]:
sst_1_behavdesign_clean

,onset,duration,trial_type,subject,wave,beta
0,0.00000,0.63061,correct-go,DEV004,wave1,beta_0001.nii
1,2.25834,0.75000,ITI,DEV004,wave1,beta_0002.nii
2,3.00834,2.00556,failed-stop,DEV004,wave1,beta_0003.nii
3,5.01390,1.37500,ITI,DEV004,wave1,beta_0004.nii
4,6.38890,0.43276,correct-go,DEV004,wave1,beta_0005.nii
...,...,...,...,...,...,...
52219,412.82572,1.75000,ITI,DEV225,wave1,beta_0252.nii
52220,414.57572,0.50949,correct-go,DEV225,wave1,beta_0253.nii
52221,417.08682,0.75000,ITI,DEV225,wave1,beta_0254.nii
52222,417.83682,1.50000,failed-stop,DEV225,wave1,beta_0255.nii


In [13]:
behavdesign = sst_1_behavdesign_clean
sst_1_behavdesign_clean_dev005 = behavdesign[[s in 'DEV005' for s in behavdesign.subject]]

In [14]:
sst_1_behavdesign_clean_dev005.groupby('trial_type').count()

,onset,duration,subject,wave,beta
trial_type,,,,,
ITI,128,128,128,128,128
correct-go,80,80,80,80,80
correct-stop,5,5,5,5,5
failed-go,16,16,16,16,16
failed-stop,27,27,27,27,27


## Continued...

In [88]:
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:2],'betaseries',
    behavioral_design=sst_1_behavdesign_clean,
    out_folder = ml_data_folderpath + "/")

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/
DEV005
...................................................................................................................................................................................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
...................................................................................................................................................................................................................................................................verified that all expected betas exist! Proc

OK, not bad, so we have learned to load data, and show it. We might be best to re-write the code, though, to use the first image as the image series's own mask--using `nilearn.masking.compute_brain_mask`, rather than worry about why the input mask isn't quite aligned. 

In [89]:
len(train_subjs)

59

In [94]:
import_sst_betaseries_w1_subjs_to_pkl(
     train_subjs[0:6],'betaseries',
    behavioral_design=sst_1_behavdesign_clean,
    out_folder = ml_data_folderpath + "/SST/")

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/
DEV005
...................................................................................................................................................................................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
...................................................................................................................................................................................................................................................................verified that all expected betas exist! Proc

In [16]:
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:40],'betaseries',
    behavioral_design=sst_1_behavdesign_clean,
    out_folder = ml_data_folderpath + "/SST/")

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/
DEV005
...................................................................................................................................................................................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
...................................................................................................................................................................................................................................................................verified that all expected betas exist! Proc

In [ ]:
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:40],'betaseries',
    behavioral_design=sst_1_behavdesign_clean,
    out_folder = ml_data_folderpath + "/SST/")

## Just subjects in the conditions of interest: CorrectStop vs. CorrectGo

In [15]:
sst_1_behavdesign_clean.trial_type.value_counts()

ITI             26112
correct-go      18750
failed-stop      3434
correct-stop     3094
failed-go         834
Name: trial_type, dtype: int64

In [16]:
sst_1_behavdesign_correct_conditions=sst_1_behavdesign_clean[sst_1_behavdesign_clean.trial_type.isin(['correct-go','correct-stop'])]

In [17]:
sst_1_behavdesign_correct_conditions.trial_type.value_counts()

correct-go      18750
correct-stop     3094
Name: trial_type, dtype: int64

In [18]:
sst_1_behavdesign_correct_conditions

,onset,duration,trial_type,subject,wave,beta
0,0.00000,0.63061,correct-go,DEV004,wave1,beta_0001.nii
4,6.38890,0.43276,correct-go,DEV004,wave1,beta_0005.nii
8,15.65280,0.69477,correct-go,DEV004,wave1,beta_0009.nii
10,20.53475,0.85534,correct-go,DEV004,wave1,beta_0011.nii
12,23.35212,0.48997,correct-go,DEV004,wave1,beta_0013.nii
...,...,...,...,...,...,...
52210,398.86111,0.51663,correct-go,DEV225,wave1,beta_0243.nii
52214,403.99723,0.72046,correct-go,DEV225,wave1,beta_0247.nii
52216,407.31460,0.62814,correct-go,DEV225,wave1,beta_0249.nii
52218,410.82016,0.51824,correct-go,DEV225,wave1,beta_0251.nii


In [19]:
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:5],'betaseries',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
out_file_suffix = '_correct_cond')

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/
DEV005
........................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.................................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0009.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
.............

In [31]:
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:40],'betaseries',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
out_file_suffix = '_correct_cond')

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/
DEV005
........................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.................................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0009.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
.............

In [19]:
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs[0:15],'betaseries',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
out_file_suffix = '_correct_cond')

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/
DEV005
........................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.................................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0009.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
.............

In [20]:
#we have data missing fro mthese subjects below. not enstirely sure yet; see 
#https://docs.google.com/document/d/1EB0ACA6qhMkDEgv-zqNTepPCe_x41fRQHaeqT47vazM/edit#heading=h.60rb13rlxkrq


train_subjs_selected = [ts for ts in train_subjs if (ts not in exclude_subjects)]

In [21]:
import_sst_betaseries_w1_subjs_to_pkl(
    train_subjs_selected,'betaseries',
    behavioral_design=sst_1_behavdesign_correct_conditions,
    out_folder = ml_data_folderpath + "/SST/",
out_file_suffix = '_correct_cond')

/gpfs/projects/sanlab/shared/DEV/nonbids_data/fMRI/fx/models/SST/wave1/betaseries/
DEV005
........................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV006
.................................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0009.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV009
.............

...imported data.
temp mask deleted
DEV027
.....................................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0003.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV028
................................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0003.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV029
................................

...imported data.
temp mask deleted
DEV053
.....................................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0003.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV055
.....................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV056
...........................................

...imported data.
temp mask deleted
DEV085
.................................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0005.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV086
.....................................................................................................................verified that all expected betas exist! Processing...setting the mask to the first image in the series beta_0001.nii
During mask creation, received error 'Resampling binary images with continuous or linear interpolation.'. This is normal.
temp mask created.
loading files. This step may take some time...
...imported data.
temp mask deleted
DEV087
...............................